# BioExplorer - Animated vasculature
![](../../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [37]:
from bioexplorer import BioExplorer, CellAnimationParams
from tqdm import tqdm
import seaborn as sns

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [38]:
def set_materials(model_id, palette_name, cast_user_data):
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    cast_user_datas = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, len(material_ids))

    for _ in material_ids:
        shading_modes.append(be.SHADING_MODE_PERLIN)
        glossinesses.append(0.1)
        user_params.append(0.05)
        specular_exponents.append(5.0)
        cast_user_datas.append(cast_user_data)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette, shading_modes=shading_modes,
        specular_exponents=specular_exponents, user_parameters=user_params,
        cast_user_datas=cast_user_datas
    )

### Load NGV system

In [39]:
neurons_population_name = 'o1'
astrocytes_population_name = 'astrocytes'
vasculature_population_name = 'vasculature'

morphology_realism_level = be.MORPHOLOGY_REALISM_LEVEL_ALL
vasculature_realism_level = be.VASCULATURE_REALISM_LEVEL_ALL
center = [264.830629,1843.324822,328.217474]
filter = 'SQRT(POW(x-%f, 2.0) + POW(y-%f, 2.0) + POW(z-%f, 2.0))<' % (center[0], center[1], center[2])

In [40]:
def add_neurons(seed, amplitude, frequency):
    neurons_assembly_name = 'Neurons'
    be.remove_assembly(neurons_assembly_name)
    neurons_assembly = be.add_assembly(neurons_assembly_name)
    neurons_model = be.add_neurons(
        assembly_name=neurons_assembly_name,
        population_name=neurons_population_name,
        morphology_representation=be.MORPHOLOGY_REPRESENTATION_SECTION,
        simulation_report_id=2,
        realism_level=morphology_realism_level,
        load_synapses=True,
        load_somas=True,
        load_axon=True, generate_varicosities=False, generate_externals=False,
        load_basal_dendrites=True, load_apical_dendrites=True,
        sql_node_filter=filter + '100',
        animation_params=CellAnimationParams(
            seed=seed, offset=100, amplitude=amplitude, frequency=frequency)
    )

In [ ]:
def add_astrocytes(seed, amplitude, frequency):
    astrocytes_assembly_name = 'Astrocytes'
    be.remove_assembly(astrocytes_assembly_name)
    vasculature_assembly = be.add_assembly(astrocytes_assembly_name)
    vasculature_model = be.add_astrocytes(
        assembly_name=astrocytes_assembly_name,
        population_name=astrocytes_population_name,
        vasculature_population_name='vasculature',
        radius_multiplier=1.0,
        realism_level=morphology_realism_level,
        sql_filter='guid%2=0 AND ' + filter + '120',
        animation_params=CellAnimationParams(
            seed=seed, offset=100, amplitude=amplitude, frequency=frequency)
    )

In [ ]:
def add_vasculature(seed, amplitude, frequency):
    vasculature_assembly_name = 'Vasculature'
    be.remove_assembly(vasculature_assembly_name)
    vasculature_assembly = be.add_assembly(vasculature_assembly_name)
    vasculature_model = be.add_vasculature(
        assembly_name=vasculature_assembly_name,
        population_name=vasculature_population_name,
        representation=be.VASCULATURE_REPRESENTATION_SEGMENT,
        realism_level=vasculature_realism_level,
        sql_filter=filter + '120',
        animation_params=CellAnimationParams(
            seed=seed, offset=100, amplitude=amplitude, frequency=frequency)
    )

In [ ]:
status = core.set_renderer(
    background_color=[0, 0, 0], head_light=True,
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.fog_start = 50.0
params.fog_thickness = 200.0
params.gi_samples = 0
params.gi_weight = 0.2
params.gi_distance = 25.0
params.shadows = 0.7
params.soft_shadows = 0.2
params.epsilon_factor = 1.0
params.max_bounces = 1
params.show_background = False
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
def set_camera():
    core.set_camera(
        current='bio_explorer_perspective',
        orientation=[-0.8403140268747967, -0.4224086257302892, 0.1860602052612373, 0.2842971845872873],
        position=[167.03493041290807, 1850.980448577644, 261.049429564132],
        target=[209.7247427525868, 1826.2248206861336, 320.4356904127371],
    )
    params = core.PerspectiveCameraParams()
    params.aperture_radius = 0
    params.focus_distance = 66.24
    params.fovy = 45.0
    core.set_camera_params(params)

In [ ]:
from bioexplorer import MovieMaker
import seaborn as sns

mm = MovieMaker(be)

amplitude=0.2
frequency=0.1
palettes = ['GnBu_r', 'Wistia', 'Reds_r']

set_camera()

for frame in tqdm(range(0, 1800)):
    seed = frame + 1
    be.reset_scene()
    '''NGV'''
    add_neurons(seed=seed, amplitude=amplitude, frequency=frequency)
    add_astrocytes(seed=seed, amplitude=amplitude, frequency=frequency)
    add_vasculature(seed=seed, amplitude=amplitude, frequency=frequency)

    '''Simulation'''
    core.set_animation_parameters(current=(frame * 5) % 1750)

    '''Materials'''
    model_ids = be.get_model_ids()['ids']
    palette = 0
    for model_id in model_ids:
        cast_user_data = False
        if palette==0:
            cast_user_data = True
        set_materials(model_id, palettes[palette], cast_user_data)
        palette += 1

    '''Snapshot'''
    mm.create_snapshot(
        renderer='advanced',
        path='/tmp', base_name='%05d' % frame,
        size=[960, 540], samples_per_pixel=64)